ESSE PROJETO JÁ É DIFERENTE NÃO É UMA API, MAS SIM BUSCANDO DADOS DE UM SITE https://www.residentevildatabase.com/personagens/ada-wong/ E USANDO O INSPECIONAR OU F12

SERIA ALGO PARECIDO COM O SELENIUN QUE FAZ PASSO A PASSO O CAMINHO DO SEU MOUSE

https://curlconverter.com/

SITES CONSTRUIDOS EM JAVA SCRIPT PODE SER MAIS COMPLEXOS

In [0]:
import requests  # Importa a biblioteca para fazer requisições HTTP
%pip install beautifulsoup4
from bs4 import BeautifulSoup  # Importa a biblioteca para parsear HTML
import pandas as pd

## PARA PEGAR O CÓDIGO HTML DO SITE BASTA IR EM FERRAMENTAS DO DEV OU F12 IR EM NETWORK E DAR CTRL R E COPIAR AS CURL
## APÓS PROCURAR NO GOOGLE CURL TO PYTHON PARA CONVERTER O COPIADO ACIMA

# Definição do cabeçalho HTTP para simular um navegador real
HEADERS = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'max-age=0',
    'referer': 'https://www.residentevildatabase.com/personagens/',
    'sec-ch-ua': '"Not(A:Brand";v="99", "Google Chrome";v="133", "Chromium";v="133"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36'
}

def get_content(url):
    """
    Faz uma requisição HTTP para a URL fornecida e retorna a resposta.
    """
    try:
        resp = requests.get(url, headers=HEADERS, timeout=10)  # Timeout para evitar travamento
        return resp
    except requests.RequestException:
        print(f"Erro ao acessar: {url}")
        return None  # Retorna None em caso de erro na requisição

def get_basic_infos(soup):
    """
    Extrai informações básicas da página HTML.
    """
    div_page = soup.find("div", class_="td-page-content")
    if not div_page:
        return {}

    paragrafo = div_page.find_all("p")[1]
    em = paragrafo.find_all("em")

    data = {}
    for i in em:
        if ":" in i.text:
            chave, valor = i.text.split(":", 1)  # Divide apenas na primeira ocorrência de ':'
            data[chave.strip()] = valor.strip()

    return data

def get_aparicoes(soup):
    """
    Extrai lista de aparições do personagem nos filmes e jogos.
    """
    h4 = soup.find("div", class_="td-page-content").find("h4")
    if not h4:
        return []

    ul = h4.find_next_sibling("ul")  # Melhor para encontrar a lista correta
    if not ul:
        return []

    aparicoes = [i.text.strip() for i in ul.find_all("li")]
    return aparicoes

def get_personagem_infos(url):
    """
    Obtém informações do personagem a partir da URL fornecida.
    """
    resp = get_content(url)
    if resp is None or resp.status_code != 200:
        print(f"Página removida ou erro ao acessar: {url}")
        return None  # Retorna None se a página não existir

    try:
        soup = BeautifulSoup(resp.text, "html.parser")  # Parseia o HTML da página
        infos = get_basic_infos(soup)  # Extrai informações básicas
        infos["Aparicoes"] = get_aparicoes(soup)  # Extrai lista de aparições
        return infos
    except Exception as e:
        print(f"Erro ao processar {url}: {e}")
        return None  # Retorna None se der erro ao processar os dados

def get_links():
    """
    Obtém os links de todos os personagens na página principal.
    """
    url = "https://www.residentevildatabase.com/personagens/"
    resp = requests.get(url, headers=HEADERS)

    if resp.status_code != 200:
        print("Erro ao acessar a página de personagens.")
        return []

    soup_personagens = BeautifulSoup(resp.text, "html.parser")  # Especificando o parser corretamente
    link_a = soup_personagens.find("div", class_="td-page-content").find_all("a", href=True)

    links = [i["href"] for i in link_a if i["href"].startswith("https")]
    return links

# Obtém os links de todos os personagens
links = get_links()

# Coleta as informações de cada personagem
data = []
for link in links:
    personagem_info = get_personagem_infos(link)
    if personagem_info is not None:  # Só adiciona se os dados forem válidos
        personagem_info["Link"] = link
        nome = link.strip("/").split("/")[-1].replace(".html", "").replace("-", " ").title()
        personagem_info["Nome"] = nome
        data.append(personagem_info)

# Converte os dados para um DataFrame do Pandas
df = pd.DataFrame(data)

# Exibe os primeiros resultados
display(df)



In [0]:

df = df.explode("Aparicoes")

display(df)

In [0]:
from pyspark.sql import SparkSession

# Converter o DataFrame Pandas para um DataFrame Spark
# Além disso, substitui espaços nos nomes das colunas por underscores (_)
df_spark = spark.createDataFrame(df).toDF(*[c.replace(' ', '_') for c in df.columns])

# Salvar a tabela no banco de dados 'resident_evil' no formato Delta
# O nome da tabela parece estar repetido ('resident_evil.resident_evil.resident_evil_bronze'),
# o que pode causar erro. Normalmente, o correto seria "database.tabela" (ex: 'resident_evil.resident_evil_bronze').
df_spark.write.mode("overwrite").format("delta").saveAsTable("resident_evil.resident_evil.resident_evil_bronze")


In [0]:
%sql

SELECT * FROM resident_evil.resident_evil.resident_evil_bronze